In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython import display
import time

plt.rcParams['figure.figsize'] = [20, 10]
plt.rcParams['lines.markersize'] = 10
fig = plt.figure()

plt.plot()

In [5]:
def init_params(points):
    if points >= 0: # Change the layers
        layers = [2, 50, 25, 10, 1]
    else:
        layers = [2, 50, 50, 1]
    w = []
    for i in range(len(layers)-1):
        w.append(np.random.randn(layers[i+1], layers[i]))
    b = []
    for i in w:
        b.append(np.random.randn(len(i.T[0]), 1))
    lr = 0.1
    return w, b, lr

sigmoid = lambda x: 1/(1+np.exp(-x))
der_sigmoid = lambda x: x*(1-x)

def sign(x):
    lx = []
    for i in x:
        if i[0] > 0:
            lx.append([1])
        else:
            lx.append([-1])
    return np.array(lx)

def get_guess(w, b, l0):
    l = [l0]
    for i in range(len(w)-1):
        # l.append(relu(w[i].dot(l[i]) + b[i]))
        l.append(sigmoid(w[i].dot(l[i]) + b[i]))
    l.append(sign(w[-1].dot(l[-1]) + b[-1]))
    return l

def train(w, b, l, target, lr):
    d = [target - l[-1]]
    for i in range(len(l)-2):
        # d.append(der_relu(w[-(1+i)].T.dot(d[-1]), l[-(2+i)]))
        # d.append(der_sigmoid(w[-(1+i)].T.dot(d[-1]), l[-(2+i)]))
        d.append(w[-(1+i)].T.dot(d[-1]) * der_sigmoid(l[-(2+i)]))
    for i in range(len(w)):
        w[-(1+i)] += d[i] * l[-(2+i)].T * lr
        b[-(1+i)] += d[i] * lr

In [10]:
points = 50 # change the amount of points
x = np.random.random(points)
y = np.random.random(points)
lb = []

fig.clf()
img1 = fig.add_subplot(131)
img1.set_aspect('equal')
# img1.axis([-0.2,1.2,-0.2,1.2])

for i in range(len(x)):
    n = np.random.choice([-1, 1])
    # n = int(x[i]>y[i])*2-1
    lb.append(n)
    if n == 1:
        img1.plot(x[i], y[i], 'co')
    else:
        img1.plot(x[i], y[i], 'mo')

img2 = fig.add_subplot(132)
img2.set_aspect('equal')
# img2.axis([0,1,0,1])

w, b, lr = init_params(points)
iters = 0
hst_acc = 0
is_done = False
# time_start = time.time()
while not is_done:
    img2.clear()
    acc = 0
    iters += 1
    for i in range(len(x)):
        l0 = np.array([[x[i]], [y[i]]])
        target = lb[i]
        l = get_guess(w, b, l0)
        if l[-1] == target:
            img2.plot(l0[0], l0[1], 'go')
            acc += 1
        else:
            img2.plot(l0[0], l0[1], 'ro')
        train(w, b, l, target, lr)
    hst_acc = np.maximum(hst_acc, acc/len(x))
    # if hst_acc >= 1-(np.floor((time.time()-time_start)/1200)*0.05):
    if hst_acc >= 1-(np.floor(iters/10000)*0.05) and hst_acc == acc/len(x): # set the error margin per iteration
        is_done = True
    if iters % 100 == 0: # Change the log occurence
        show_graphs = False # Set if the graph apper every N iteration
        if show_graphs:
            display.display('Iterations: '+str(iters)+' - Accuracy: '+str(acc/len(x))+' ({})'.format(hst_acc), fig)
        else:
            print('Iterations: '+str(iters)+' - Accuracy: '+str(acc/len(x))+' ({})'.format(hst_acc))
        display.clear_output(wait=True)

print('Iterations: '+str(iters)+' - Accuracy: '+str(acc/len(x))+'\nLoading viewing map...')
display.clear_output(wait=True)

show_map = True
if show_map:
    img3 = fig.add_subplot(133)
    img3.set_aspect('equal')

    xx = np.linspace(0,1,50)
    yy = np.linspace(0,1,50)
    for i in xx:
        for k in yy:
            l = get_guess(w, b, np.array([[i], [k]]))
            if l[-1] == 1:
                img3.plot(i, k, 'c.')
            else:
                img3.plot(i, k, 'm.')

print('Iterations: '+str(iters)+' - Accuracy: '+str(acc/len(x)))
fig

KeyboardInterrupt: 

In [ ]:
img3 = fig.add_subplot(133)
img3.set_aspect('equal')

xx = np.linspace(0,1,50)
yy = np.linspace(0,1,50)
for i in xx:
    for k in yy:
        l = get_guess(w, b, np.array([[i], [k]]))
        if l[-1] == 1:
            img3.plot(i, k, 'c.')
        else:
            img3.plot(i, k, 'm.')
fig